In [ ]:
#pipeline pseudocode
init_pipeline(params)
    #prepare_image_and_dirs - at now we manually put fits files in current dir
    init_astromatic_wrapper # create kwargs dicts for astromatic objects from global params

# assume we have some images image1.fits image2.fits image3.fits and corresponding to them bias dark and flat fits
# 1 pre-reduction of FITS (bias, dark ,flat - HOW???) output - pr_images
# 2 calibrate images using SCAMP (which uses SExtractor catalog)
    # create catalog from pre-reduced fits
    sextractor1 = aw.api.Astromatic(**sex_kwargs_1)
    sextractor1.run(pr_image1.fits) # output - sex_output_pr_1
    # send catalog to SCAMP - it will calculate astrometry and photometry solution (write into header)
    scamp = aw.api.Astromatic(**scamp_kwargs)
    scamp.run(sex_output_pr_1.fits) # output - calibrated_pr_1
    # for 2 and 3 the same way
# 3 co-addition and resampling of calibrated pre-reduced images
    swarp = aw.api.Astromatic(**swarp_kwargs)
    swarp.run(calibrated_pr_images) # output - preprocessed_coadd
# 4 extracting sources from preprocessed images
    sextractor2 = aw.api.Astromatic(**sex_kwargs_2)
    sextractor2.run(preprocessed_coadd.fits) # output - catalog_1
    psfex = aw.api.Astromatic(**psfex_kwargs)
    psfex.run(catalog_1) # output - psf_coadd
    sextractor3 = aw.api.Astromatic(**sex_kwargs_3)
    sextractor3.run(preprocessed_coadd.fits, psf_coadd.fits) # output - catalog_2
# catalog_2 - catalog of objects extracted from pre-reducted, preprocessed and co-added and psf-modeled fits
# futher processing:
    # a) put catalog into DB
    # b) add single images processing (not only co-added) - add to catalog objects wchich dissapeared after co-addition

# First, we check workability of components of astromatic software through astromatic_wrapper api

## Prepare invironment

In [1]:
import os
import datetime
import fnmatch
import re
import astromatic_wrapper as aw #TODO: install aw to itaf virtual env
from astropy.io import fits

In [2]:
#!pip install apex

In [3]:
import pyraf.iraf as iraf
import pyraf

In [4]:
import sys
sys.path.insert(0, '/home/ser/Dev/astro_engine/code')
from astro_utils import AEDirsTreeConfigurer

In [7]:
from fits_utils import *

In [5]:
print_iraf_tasks = 0
iraf.module.images(_doprint=print_iraf_tasks) # print or not to print (why it not working?)

Just test that PyRAF works

In [6]:
print iraf.module.imaccess('im_3.fit') #TODO: implement access check if needed
input_fits = 'im_1.fit, im_2.fit'
output_fits = 'res.fit'
s = iraf.module.imcombine(input_fits, output_fits, combine='median', Stdout=1, Stderr=1)

0


In [75]:
dirs = AEDirsTreeConfigurer()#images_path='images/GRB130427A')
dirs.build_dirs_tree()
dirs.new_log_dir()
paths = dirs.get_paths()
paths

{'catalogs': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/catalogs',
 'config': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/config',
 'images': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images',
 'logs': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/logs',
 'stacks': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/stacks',
 'temp': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp'}

In [108]:
# set image filename
files = {'image': os.path.join(paths['images'], 'GRB130427_R60_001_003.fit')}

In [57]:
# dumped code used for hardcoded configurations
common_config = {'config_files': 
        {'scamp_required_catalog_params': os.path.join(paths['config'],'scamp_required_catalog_params.json'),
        'pipeline_config': os.path.join(paths['config'], 'pipeline_config.json')
         }}
scamp_required_catalog_params = ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 
                                           'AWIN_IMAGE', 'BWIN_IMAGE', 
                                           'ERRAWIN_IMAGE','ERRBWIN_IMAGE',  'ERRTHETAWIN_IMAGE',
                                           'ERRA_WORLD', 'ERRB_WORLD','ERRTHETA_WORLD',
                                           'XWIN_WORLD', 'YWIN_WORLD',
                                           'X_WORLD', 'Y_WORLD', 
                                           'FLUX_AUTO', 'FLUX_MAX','FLUX_RADIUS', 'FLUXERR_AUTO', 
                                           'MAG_AUTO', 'MAGERR_AUTO',
                                           # flags not supported yet (need Weightwatcher)
                                           #'IMAFLAGS_ISO', 'FLAGS_WEIGHT',
                                           'FLAGS', 'ELONGATION'] # TODO: refine this!
pipeline_config = {'INPUT_FITS_FORMAT': 'fit' ,'sex_kwargs_calibration_catalogs': sex_kwargs_calibration_catalogs}

sex_config_name = 'default.sex'
scamp_config_name = 'default.scamp'
swarp_config_name = 'default.swarp'
sex_config_file = os.path.join(paths['config'], sex_config_name)
scamp_config_file = os.path.join(paths['config'], scamp_config_name)
swarp_config_file = os.path.join(paths['config'], swarp_config_name)

filter_kernel = 'gauss_4.0_7x7.conv'
filter_name = os.path.join(paths['config'], filter_kernel)
detect_thresh = '5'
analysis_thresh = '5'
filter_ = 'Y'
params = ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'AWIN_IMAGE', 'BWIN_IMAGE',
                          'ERRAWIN_IMAGE','ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 
                          #'ERRA_WORLD', 'ERRB_WORLD', 
                          #'ERRTHETA_WORLD', 'X_WORLD', 'Y_WORLD', 'XWIN_WORLD', 'YWIN_WORLD', 
                          'FLUX_AUTO', 'FLUX_MAX', 'MAG_AUTO', 'FLUXERR_AUTO', 
                          # flags not supported yet (need Weightwatcher)
                          #'IMAFLAGS_ISO', 'FLAGS_WEIGHT',
                          'FLAGS', 'FLUX_RADIUS', 'ELONGATION']

# TODO: use full SExtractor config params list for set sex_kwargs (consider logic like use fits_ldac output, etc)
# define SExtractor kwargs that are common for all images
sex_kwargs_calibration_catalogs = {'code': 'SExtractor'}
sex_kwargs_calibration_catalogs['config_file'] = sex_config_file
sex_kwargs_calibration_catalogs['config'] = dict()
sex_kwargs_calibration_catalogs['config']['DETECT_THRESH'] = detect_thresh
sex_kwargs_calibration_catalogs['config']['ANALYSIS_THRESH'] = analysis_thresh
sex_kwargs_calibration_catalogs['config']['FILTER'] = filter_
sex_kwargs_calibration_catalogs['config']['FILTER_NAME'] = filter_name
sex_kwargs_calibration_catalogs['temp_path'] = paths['temp']
sex_kwargs_calibration_catalogs['params'] = params

scamp_kwargs_calibration_headers = {'code': 'SCAMP'}
scamp_kwargs_calibration_headers['config_file'] = scamp_config_file
scamp_kwargs_calibration_headers['config'] = dict()
scamp_kwargs_calibration_headers['config']['ASTREF_CATALOG'] = 'UCAC-3'
scamp_kwargs_calibration_headers['config']['ASTREF_BAND'] = 'DEFAULT'
scamp_kwargs_calibration_headers['config']['SOLVE_ASTROM'] = 'Y'
scamp_kwargs_calibration_headers['config']['SOLVE_PHOTOM'] = 'Y'
scamp_kwargs_calibration_headers['config']['CHECKPLOT_DEV'] = 'NULL'
scamp_kwargs_calibration_headers['config']['MOSAIC_TYPE'] = 'UNCHANGED'
#scamp_kwargs_calibration_headers['config']['DISTORT_DEGREES'] = '1'
scamp_kwargs_calibration_headers['config']['COMPUTE_PARALLAXES'] = 'N'
scamp_kwargs_calibration_headers['config']['CROSSID_RADIUS'] = '3.0'
scamp_kwargs_calibration_headers['config']['EXPOTIME_KEY'] = 'EXPTIME'
scamp_kwargs_calibration_headers['config']['PIXSCALE_MAXERR'] = '1.2'
scamp_kwargs_calibration_headers['config']['STABILITY_TYPE'] = 'INSTRUMENT'
scamp_kwargs_calibration_headers['config']['SN_THRESHOLDS'] = '10.0,100.0'
scamp_kwargs_calibration_headers['config']['FWHM_THRESHOLDS'] = '0.0,10.0' #default: '0.0,100.0'

swarp_kwargs_calibration = {'code': 'SWarp'}
swarp_kwargs_calibration['config_file'] = swarp_config_file
swarp_kwargs_calibration['config'] = dict()
swarp_kwargs_calibration['config']['RESAMPLE'] = 'N'
swarp_kwargs_calibration['config']['COMBINE'] = 'Y'
swarp_kwargs_calibration['config']['RESAMPLE_DIR'] = paths['images']
swarp_kwargs_calibration['config']['RESAMPLE_SUFFIX'] = '.resamp.fits' 
swarp_kwargs_calibration['config']['DELETE_TMPFILES'] = 'N'
swarp_kwargs_calibration['config']['PROJECTION_TYPE'] = 'NONE'
swarp_kwargs_calibration['config']['CELESTIAL_TYPE'] = 'NATIVE'

filter_kernel = 'gauss_4.0_7x7.conv' # TODO: is this filter is best here?
filter_name = os.path.join(paths['config'], filter_kernel)
detect_thresh = '3'
analysis_thresh = '3'
filter_ = 'Y'
params = ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'AWIN_IMAGE', 'BWIN_IMAGE',
                          'ERRAWIN_IMAGE','ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 
                          'ERRA_WORLD', 'ERRB_WORLD', 
                          'ERRTHETA_WORLD', 'X_WORLD', 'Y_WORLD', 'XWIN_WORLD', 'YWIN_WORLD', 
                          'FLUX_AUTO', 'FLUX_MAX', 'MAG_AUTO', 'FLUXERR_AUTO', 
                          # flags not supported yet (need Weightwatcher)
                          #'IMAFLAGS_ISO', 'FLAGS_WEIGHT',
                          'FLAGS', 'FLUX_RADIUS', 'ELONGATION']

# TODO: use full SExtractor config params list for set sex_kwargs (consider logic like use fits_ldac output, etc)
# define SExtractor kwargs that are common for all images
sex_kwargs_singe_images_catalogs = {'code': 'SExtractor'}
sex_kwargs_singe_images_catalogs['config_file'] = sex_config_file
sex_kwargs_singe_images_catalogs['config'] = dict()
sex_kwargs_singe_images_catalogs['config']['DETECT_THRESH'] = detect_thresh
sex_kwargs_singe_images_catalogs['config']['ANALYSIS_THRESH'] = analysis_thresh
sex_kwargs_singe_images_catalogs['config']['FILTER'] = filter_
sex_kwargs_singe_images_catalogs['config']['FILTER_NAME'] = filter_name
sex_kwargs_singe_images_catalogs['temp_path'] = paths['temp']
sex_kwargs_singe_images_catalogs['params'] = params

swarp_kwargs_coaddition = {'code': 'SWarp'}
swarp_kwargs_coaddition['config_file'] = swarp_config_file
swarp_kwargs_coaddition['config'] = dict()
swarp_kwargs_coaddition['config']['RESAMPLE'] = 'N'
swarp_kwargs_coaddition['config']['COMBINE'] = 'Y'
swarp_kwargs_coaddition['config']['RESAMPLE_DIR'] = paths['temp']
swarp_kwargs_coaddition['config']['RESAMPLE_SUFFIX'] = '.resamp.fits' 
swarp_kwargs_coaddition['config']['DELETE_TMPFILES'] = 'Y'
swarp_kwargs_coaddition['config']['PROJECTION_TYPE'] = 'NONE'
swarp_kwargs_coaddition['config']['CELESTIAL_TYPE'] = 'NATIVE'

filter_kernel = 'gauss_4.0_7x7.conv' # TODO: is this filter is best here?
filter_name = os.path.join(paths['config'], filter_kernel)
detect_thresh = '3'
analysis_thresh = '3'
filter_ = 'N'
params = ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'AWIN_IMAGE', 'BWIN_IMAGE',
                          'ERRAWIN_IMAGE','ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 
                          'ERRA_WORLD', 'ERRB_WORLD', 
                          'ERRTHETA_WORLD', 'X_WORLD', 'Y_WORLD', 'XWIN_WORLD', 'YWIN_WORLD', 
                          'FLUX_AUTO', 'FLUX_MAX', 'MAG_AUTO', 'FLUXERR_AUTO', 
                          # flags not supported yet (need Weightwatcher)
                          #'IMAFLAGS_ISO', 'FLAGS_WEIGHT',
                          'FLAGS', 'FLUX_RADIUS', 'ELONGATION']

# TODO: use full SExtractor config params list for set sex_kwargs (consider logic like use fits_ldac output, etc)
# define SExtractor kwargs that are common for all images
sex_kwargs_coadded_image_catalog = {'code': 'SExtractor'}
sex_kwargs_coadded_image_catalog['config_file'] = sex_config_file
sex_kwargs_coadded_image_catalog['config'] = dict()
sex_kwargs_coadded_image_catalog['config']['DETECT_THRESH'] = detect_thresh
sex_kwargs_coadded_image_catalog['config']['ANALYSIS_THRESH'] = analysis_thresh
sex_kwargs_coadded_image_catalog['config']['FILTER'] = filter_
sex_kwargs_coadded_image_catalog['config']['FILTER_NAME'] = filter_name
sex_kwargs_coadded_image_catalog['temp_path'] = paths['temp']
sex_kwargs_coadded_image_catalog['params'] = params

<h2>Pipeline (local version)</h2>

<h3>Import all needed packages</h3>

In [2]:
import os
import datetime
import copy
import astromatic_wrapper as aw #TODO: install aw to itaf virtual env
from astropy.io import fits
import pyraf.iraf as iraf
import pyraf
import sys
sys.path.insert(0, '/home/ser/Dev/astro_engine/code')
from astro_utils import AEDirsTreeConfigurer, AEJsonConfigLoader
from fits_utils import *

<h3>Define pipeline parameters</h3>

Define dirs tree

In [3]:
dirs = AEDirsTreeConfigurer()#images_path='images/GRB130427A')
dirs.build_dirs_tree()
dirs.new_log_dir()
paths = dirs.get_paths()
paths

{'catalogs': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/catalogs',
 'config': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/config',
 'images': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images',
 'logs': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/logs',
 'stacks': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/stacks',
 'temp': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp'}

Define config for astro software from json config files

In [4]:
all_configs = AEJsonConfigLoader(os.path.join(paths['config'], 'common_config.json'))
print all_configs.get_required_common_params()
all_configs.build_config()
pipeline_config = all_configs.get_pipeline_config()

{'config_files': ['pipeline_config']}


In [5]:
for key in pipeline_config['pipeline_stages'].keys():
    print "-----",key
    for kkey in pipeline_config['pipeline_stages'][key]['stage_steps'].keys():
        print kkey
    print '-----------'

----- preparation
-----------
----- images_calibration
create_calibration_headers
basic_reduction
create_calibrated_images_catalogs
calibrate_images
model_psf_calibrated_images_catalogs
create_calibration_catalogs
create_calibrated_images_catalogs_psf
-----------
----- images_coaddition
model_psf_coadded_image_catalog
coadd_calibrated_images
create_coadded_image_catalog
create_coadded_image_catalog_psf
-----------


<h4>Catalog extraction basic function</h4>

In [13]:
def create_catalogs(input_images_names, paths, pipeline_config, pipeline_stage, pipeline_step):
    # TODO: check config, stage and step for needed for this function keys
    # select pipeline stage and step in config
    step_config = copy.deepcopy(pipeline_config['pipeline_stages'][pipeline_stage]['stage_steps'][pipeline_step])
    step_params_config = step_config['step_params']
    
    # use paths to fill path-depended params TODO: preprocess config to do this
    step_params_config['software_params']['config_file'] = os.path.join(paths['config'],
                                                                step_params_config['software_params']['config_file'])
    step_params_config['software_params']['temp_path'] = paths[step_params_config['software_params']['temp_path']]
    step_params_config['software_params']['config']['FILTER_NAME'] = os.path.join(paths['config'],
                                                      step_params_config['software_params']['config']['FILTER_NAME'])
    
    # prepare io params   
    # input images
    input_images_format = to_ext_format(step_params_config['input_params']['images_format'])
    input_images_dir = paths[step_params_config['input_params']['images_dir']]
    iffs = build_filenames_list(input_images_names, new_ext=input_images_format, 
                                new_path=input_images_dir)
    
    # input psf models, if exist
    look_for_psf = False
    ipfs = range(len(iffs))
    if 'psf_models_format' in step_params_config['input_params'].keys() and \
    'psf_models_dir' in step_params_config['input_params'].keys():
        look_for_psf = True
        input_psf_models_format = to_ext_format(step_params_config['input_params']['psf_models_format'])
        input_psf_models_dir = paths[step_params_config['input_params']['psf_models_dir']]
        ipfs = build_filenames_list(input_images_names, new_ext=input_psf_models_format, 
                                    new_path=input_psf_models_dir)
    
    # output catalogs
    output_catalogs_format = to_ext_format(step_params_config['output_params']['catalogs_format'])
    output_catalogs_dir = paths[step_params_config['output_params']['catalogs_dir']]
    ocfs = build_filenames_list(input_images_names, new_ext=output_catalogs_format, 
                                    new_path=output_catalogs_dir)
    
    
    if 'check_params_for_scamp_required' in step_params_config['output_params'].keys():
        if step_params_config['output_params']['check_params_for_scamp_required'] == 'true':
            missing_req_scamp_params = list(set(pipeline_config['scamp_required_catalog_params']) - \
            set(step_params_config['software_params']['params']))
            step_params_config['software_params']['params'].extend(missing_req_scamp_params)
    
    if 'check_params_for_psfex_required' in step_params_config['output_params'].keys():
        if step_params_config['output_params']['check_params_for_psfex_required'] == 'true':
            missing_req_psfex_params = list(set(pipeline_config['psfex_required_catalog_params']) - \
            set(step_params_config['software_params']['params']))
            step_params_config['software_params']['params'].extend(missing_req_psfex_params)
    
    # output is a list of catalogs full pathnames
    catalogs = []
    
    for (iff, ipf, ocf) in zip(iffs, ipfs, ocfs):
        software_params = step_params_config['software_params'] # TODO: deepcopy or not?
        # TODO: is it needed or SExtractor recognizes it by itself?
        if 'params_from_fits_header' in step_params_config['data_handle'].keys():
            fits_file = fits.open(iff)
            fits_file_header = fits_file[0].header
            for param_name, fits_header_keyword in step_params_config['data_handle']['params_from_fits_header'].iteritems():
                if fits_header_keyword in fits_file_header:
                    software_params['config'][param_name] = str(fits_file[0].header[fits_header_keyword])
        software_params['config']['CATALOG_NAME'] = ocf
        if look_for_psf:
            software_params['config']['PSF_NAME'] = ipf
        sextractor = aw.api.Astromatic(**software_params)
        #this_cmd, kwargs2 = sextractor.build_cmd(iff, **software_params)
        #print this_cmd
        sextractor.run(iff) #TODO: handle run status (maybe return it?)
        catalogs.append(ocf)
    
    return catalogs

<h4>Calculate calibration basic function</h4>

In [14]:
def calculate_calibration(input_images_names, paths, pipeline_config, pipeline_stage, pipeline_step):
    # TODO: check config, stage and step for needed for this function keys
    # select pipeline stage and step in config
    step_config = copy.deepcopy(pipeline_config['pipeline_stages'][pipeline_stage]['stage_steps'][pipeline_step])
    step_params_config = step_config['step_params']
    
    # use paths to fill path-depended params TODO: preprocess config to do this
    step_params_config['software_params']['config_file'] = os.path.join(paths['config'],
                                                                step_params_config['software_params']['config_file'])
    step_params_config['software_params']['temp_path'] = paths[step_params_config['software_params']['temp_path']]
    
    # prepare io params   
    # input catalogs
    input_catalogs_format = to_ext_format(step_params_config['input_params']['catalogs_format'])
    input_catalogs_dir = paths[step_params_config['input_params']['catalogs_dir']]
    icfs = build_filenames_list(input_images_names, new_ext=input_catalogs_format, 
                                    new_path=input_catalogs_dir)
    
    # output is empty list
    result = []
    
    calculate_calibration_separately = False
    if 'calculate_calibration_separately' in step_params_config['output_params'].keys():
        if step_params_config['output_params']['calculate_calibration_separately'] == 'true':
            calculate_calibration_separately = True
            
    software_params = step_params_config['software_params'] # TODO: deepcopy or not?
    
    if calculate_calibration_separately:
        for icf in icfs:
            scamp = aw.api.Astromatic(**software_params)
            #this_cmd, kwargs2 = scamp.build_cmd(icf, **software_params)
            #print this_cmd
            scamp.run(icf)
    else:
        scamp = aw.api.Astromatic(**software_params)
        scamp.run(' '.join(icfs)) # TODO: create file with catalogs list, and pass as @catalogs_list
        
    return result

<h4>Image SWarp basic function</h4>

In [15]:
from operator import itemgetter
from itertools import *
def swarp_images(input_images_names, paths, pipeline_config, pipeline_stage, pipeline_step):
    # TODO: check config, stage and step for needed for this function keys
    # select pipeline stage and step in config
    step_config = copy.deepcopy(pipeline_config['pipeline_stages'][pipeline_stage]['stage_steps'][pipeline_step])
    step_params_config = step_config['step_params']
    
    # use paths to fill path-depended params TODO: preprocess config to do this
    step_params_config['software_params']['config_file'] = os.path.join(paths['config'],
                                                                step_params_config['software_params']['config_file'])
    step_params_config['software_params']['temp_path'] = paths[step_params_config['software_params']['temp_path']]
    step_params_config['software_params']['config']['RESAMPLE_DIR'] = paths[step_params_config['software_params']['config']['RESAMPLE_DIR']]
    
    # prepare io params
    # input images
    input_images_format = to_ext_format(step_params_config['input_params']['images_format'])
    input_images_dir = paths[step_params_config['input_params']['images_dir']]
    iffs = build_filenames_list(input_images_names, new_ext=input_images_format, 
                                new_path=input_images_dir)
    
    # output images
    output_images_format = to_ext_format(step_params_config['output_params']['images_format'])
    output_images_dir = paths[step_params_config['output_params']['images_dir']]
    
    # output weightmaps
    output_weightmaps_format = to_ext_format(step_params_config['output_params']['weightmaps_format'])
    output_weightmaps_dir = paths[step_params_config['output_params']['weightmaps_dir']]
    
    # output is a tuple of lists: swarped images full pathnames and weightmaps full pathnames
    swarped_images, weightmaps = [], []
    
    swarp_separately = False
    if 'swarp_separately' in step_params_config['output_params'].keys():
        if step_params_config['output_params']['swarp_separately'] == 'true':
            swarp_separately = True
            
    software_params = step_params_config['software_params'] # TODO: deepcopy or not?
                   
    if swarp_separately:
        # prepare output_images_filenames and output_weightmaps_filenames (convert them to full pathname)
        offs = build_filenames_list(input_images_names, new_ext=output_images_format, 
                                    new_path=output_images_dir)
        owmfs = build_filenames_list(input_images_names, new_ext=output_weightmaps_format, 
                                    new_path=output_weightmaps_dir)
        
        for (iff, off, owmf) in zip(iffs, offs, owmfs):
            software_params['config']['IMAGEOUT_NAME'] = off
            software_params['config']['WEIGHTOUT_NAME'] = owmf
            swarp = aw.api.Astromatic(**software_params)
            swarp.run(iff)
            swarped_images.append(off)
            weightmaps.append(owmf)   
    else:
        # build coadded image and weightmap names
        output_filename = pipeline_config['observation_id']
        coadd_images_nums = sorted(
            [os.path.basename(iff).rstrip(input_images_format).rsplit('_',1)[1] for iff in iffs])
        nums_ranges = []
        for k, g in groupby(enumerate(coadd_images_nums), lambda (i,x):int(i)-int(x)):
            nums_range = map(itemgetter(1),g)
            nums_ranges.append(nums_range[0] if len(nums_range) == 1 else nums_range[0] + '-' + nums_range[-1])
        nums_ranges = '_'.join(nums_ranges)
        output_image_name = output_filename + '_' + nums_ranges + output_images_format
        output_image_name = os.path.join(output_images_dir, output_image_name)
        output_weightmap_name = output_filename + '_' + nums_ranges + output_weightmaps_format
        output_weightmap_name = os.path.join(output_weightmaps_dir, output_weightmap_name)
        # run SWarp
        software_params['config']['IMAGEOUT_NAME'] = output_image_name
        software_params['config']['WEIGHTOUT_NAME'] = output_weightmap_name
        swarp = aw.api.Astromatic(**software_params)
        swarp.run(' '.join(iffs)) # TODO: create file with images list, and pass as @images_list
        swarped_images.append(output_image_name)
        weightmaps.append(output_weightmap_name)
    
    return swarped_images, weightmaps

<h4>Calculate PSF basic function</h4>

In [20]:
def calculate_psf(input_images_names, paths, pipeline_config, pipeline_stage, pipeline_step):
    # TODO: check config, stage and step for needed for this function keys
    # select pipeline stage and step in config
    step_config = copy.deepcopy(pipeline_config['pipeline_stages'][pipeline_stage]['stage_steps'][pipeline_step])
    step_params_config = step_config['step_params']
    
    # use paths to fill path-depended params TODO: preprocess config to do this
    step_params_config['software_params']['config_file'] = os.path.join(paths['config'],
                                                                step_params_config['software_params']['config_file'])
    step_params_config['software_params']['temp_path'] = paths[step_params_config['software_params']['temp_path']]
    
    # prepare io params   
    # input catalogs
    input_catalogs_format = to_ext_format(step_params_config['input_params']['catalogs_format'])
    input_catalogs_dir = paths[step_params_config['input_params']['catalogs_dir']]
    icfs = build_filenames_list(input_images_names, new_ext=input_catalogs_format, 
                                    new_path=input_catalogs_dir)
    
    # output psf models
    output_psf_models_format = to_ext_format(step_params_config['output_params']['psf_models_format'])
    output_psf_models_dir = paths[step_params_config['output_params']['psf_models_dir']]
    # TODO: ispect BUG in PSFEx, it replaces PSF_SUFFIX only last extension part (after last .)
    step_params_config['software_params']['config']['PSF_SUFFIX'] = ext_split(output_psf_models_format)[-1]
    step_params_config['software_params']['config']['PSF_DIR'] = output_psf_models_dir
    opmfs = build_filenames_list(input_images_names, new_ext=output_psf_models_format, 
                                    new_path=output_psf_models_dir)
    
    # output is list of calculated psf models full pathnames
    #
    
    calculate_psf_separately = False
    if 'calculate_psf_separately' in step_params_config['output_params'].keys():
        if step_params_config['output_params']['calculate_psf_separately'] == 'true':
            calculate_psf_separately = True
            
    software_params = step_params_config['software_params'] # TODO: deepcopy or not?
    
    if calculate_psf_separately:
        for icf in icfs:
            psfex = aw.api.Astromatic(**software_params)
            #this_cmd, kwargs2 = psfex.build_cmd(icf, **software_params)
            #print this_cmd
            psfex.run(icf)
    else:
        psfex = aw.api.Astromatic(**software_params)
        #this_cmd, kwargs2 = psfex.build_cmd(' '.join(icfs), **software_params)
        #print this_cmd
        psfex.run(' '.join(icfs)) # TODO: create file with catalogs list, and pass as @catalogs_list
    
    return opmfs

<h3>Test part of pipeline (no basic reduction)</h3>

Input images are saved to temp folder, beause futher processing needs images and .head files in same directory.<br>
Define list of input images, raw and tech frames

In [25]:
input_images_names = sorted(get_fits_images_from_dir(paths['temp'], pipeline_config['input_images_format'], True, True))
input_images_names = [change_file_extension(iin, '') for iin in input_images_names]
input_images_names

['GRB130427_R60_001_001',
 'GRB130427_R60_001_002',
 'GRB130427_R60_001_003',
 'GRB130427_R60_001_004',
 'GRB130427_R60_001_005',
 'GRB130427_R60_001_006',
 'GRB130427_R60_001_007',
 'GRB130427_R60_001_008',
 'GRB130427_R60_001_009',
 'GRB130427_R60_001_010',
 'GRB130427_R60_001_011',
 'GRB130427_R60_001_012',
 'GRB130427_R60_001_013',
 'GRB130427_R60_001_014',
 'GRB130427_R60_001_015',
 'GRB130427_R60_001_016',
 'GRB130427_R60_001_017',
 'GRB130427_R60_001_018',
 'GRB130427_R60_001_019',
 'GRB130427_R60_001_020',
 'GRB130427_R60_001_021',
 'GRB130427_R60_001_022',
 'GRB130427_R60_001_023',
 'GRB130427_R60_001_024',
 'GRB130427_R60_001_025',
 'GRB130427_R60_001_026',
 'GRB130427_R60_001_027',
 'GRB130427_R60_001_028',
 'GRB130427_R60_001_029',
 'GRB130427_R60_001_030',
 'GRB130427_R60_001_031',
 'GRB130427_R60_001_032',
 'GRB130427_R60_001_033',
 'GRB130427_R60_001_034',
 'GRB130427_R60_001_035',
 'GRB130427_R60_001_036',
 'GRB130427_R60_001_037',
 'GRB130427_R60_001_038',
 'GRB130427_

Pipeline basic functions calls

In [26]:
# stage: images_calibration
catalogs_1 = create_catalogs(input_images_names, paths, pipeline_config, 'images_calibration', 'create_calibration_catalogs')
calculate_calibration(input_images_names, paths, pipeline_config, 'images_calibration', 'create_calibration_headers')
swarped_images_1, _ = swarp_images(input_images_names, paths, pipeline_config, 'images_calibration', 'calibrate_images')
catalogs_2 = create_catalogs(input_images_names, paths, pipeline_config, 'images_calibration', 'create_calibrated_images_catalogs')
calculate_psf(input_images_names, paths, pipeline_config, 'images_calibration', 'model_psf_calibrated_images_catalogs')
catalogs_3 = create_catalogs(input_images_names, paths, pipeline_config, 'images_calibration', 'create_calibrated_images_catalogs_psf')
# stage: images_coaddition
swarped_images_2, _ = swarp_images(input_images_names, paths, pipeline_config, 'images_coaddition', 'coadd_calibrated_images')
catalogs_4 = create_catalogs(swarped_images_2, paths, pipeline_config, 'images_coaddition', 'create_coadded_image_catalog')
calculate_psf(swarped_images_2, paths, pipeline_config, 'images_coaddition', 'model_psf_coadded_image_catalog')
catalogs_5 = create_catalogs(swarped_images_2, paths, pipeline_config, 'images_coaddition', 'create_coadded_image_catalog_psf')

Check coadded image catalog

In [27]:
catalog = aw.utils.ldac.get_table_from_ldac(catalogs_5[0])
catalog

NUMBER,EXT_NUMBER,XWIN_IMAGE,YWIN_IMAGE,ERRAWIN_IMAGE,ERRBWIN_IMAGE,ERRTHETAWIN_IMAGE,XWIN_WORLD,YWIN_WORLD,FLUX_AUTO,FLUXERR_AUTO,FLAGS,FLAGS_WEIGHT,FLUX_RADIUS,ELONGATION,MAG_AUTO,MAGERR_AUTO,ALPHAPSF_SKY,DELTAPSF_SKY,ERRX2PSF_WORLD,ERRY2PSF_WORLD,FLUX_PSF,FLUXERR_PSF,MAG_PSF,MAGERR_PSF,FLUXERR_APER,FLUX_APER,"VIGNET [20,20]",SNR_WIN
,,pix,pix,pix,pix,deg,deg,deg,ct,ct,,,pix,,mag,mag,deg,deg,deg2,deg2,ct,ct,mag,mag,ct,ct,ct,
int32,int16,float64,float64,float32,float32,float32,float64,float64,float32,float32,int16,int16,float32,float32,float32,float32,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32
1,1,192.032860444,37.5937114852,0.00696158,0.00693273,-8.26524,173.161560575,27.645088665,1325.46,4.21555,7,0,3.08848,1.14423,-7.80592,0.00345395,173.161501498,27.6450891282,0.0,0.0,0.0,0.0,99.0,99.0,1.17734,471.135,-1e+30 .. -0.0273438,536.145
2,1,157.185136853,11.7670700648,0.0445368,0.0443724,-74.0429,173.172049361,27.638201269,22.0547,1.23216,4,0,0.996008,2.47337,-3.35875,0.0606732,173.171982912,27.6381233894,0.0,0.0,0.0,0.0,99.0,99.0,1.14135,23.7855,-11.707 .. 0.0546875,29.7808
3,1,442.747390162,10.0890597982,0.0615729,0.0092936,59.2769,173.086090863,27.6377261226,11.2168,0.680413,4,0,0.637429,4.9223,-2.62467,0.065877,173.086056519,27.6377350076,0.0,0.0,0.0,0.0,99.0,99.0,1.13321,10.0757,-1e+30 .. -0.357422,21.3281
4,1,384.814022604,11.0132535451,0.0140614,0.0139159,-20.6614,173.10352954,27.6379824929,55.7656,0.733224,4,0,0.919027,4.37007,-4.36592,0.0142791,173.103593616,27.6380207628,0.0,0.0,0.0,0.0,99.0,99.0,1.14425,57.1465,-1.27734 .. -0.148438,87.0516
5,1,311.027260585,9.11952029695,0.0209234,0.0206898,4.26474,173.125740575,27.6374868744,30.0039,0.814952,4,0,0.879282,3.63897,-3.69294,0.0294974,173.125636194,27.6376213193,0.0,0.0,0.0,0.0,99.0,99.0,1.14187,29.7991,-1e+30 .. 0.634766,55.7287
6,1,112.10243061,10.0173770563,0.0271994,0.027104,77.0698,173.185619803,27.6377341457,27.9609,0.996414,4,0,0.951352,1.86589,-3.61638,0.0387006,173.185446527,27.6377574681,0.0,0.0,0.0,0.0,99.0,99.0,1.13812,26.5386,-1e+30 .. -0.265625,46.2193
7,1,31.5790008103,11.2711067761,0.0401451,0.0150145,5.35065,173.209858709,27.6380643646,15.9121,0.890501,4,0,0.902236,3.0866,-3.00432,0.0607766,173.209936786,27.637908046,0.0,0.0,0.0,0.0,99.0,99.0,1.13609,16.5333,-6.33594 .. -0.078125,27.4335
8,1,454.029018039,8.00243542871,0.0312204,0.00503261,-45.0,173.082695259,27.6371674438,28.5527,0.928072,20,0,0.584051,4.00667,-3.63912,0.0352991,173.082680873,27.6371462884,0.0,0.0,0.0,0.0,99.0,99.0,1.13806,25.8459,-1e+30 .. 0.439453,61.5421


### 1) SExtractor

In [109]:
fits_file = fits.open(files['image'])
# deg per pixel, need arcsecs per pixel, so multiply by 3600
pixel_scale = 0.761#fits_file[0].header['CDELT1'] * 3600.0 
fwhm = fits_file[0].header['FWHM_DEF'] # arcsecs
gain = fits_file[0].header['GAIN']
exp_time = fits_file[0].header['EXPTIME']
print pixel_scale, fwhm, gain, exp_time
print fwhm / pixel_scale
#fits_file[0].header

0.761 3.0 4.40000009537 60.0
3.94218134034


In [110]:
# set params for sextractor execution
# set code type according to sextractor
sex_kwargs_1 = {'code': 'SExtractor'}
# set path to config file
sex_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.sex')
# change some parameters in config (will not change config file set above)
sex_kwargs_1['config'] = {'CATALOG_NAME': os.path.join(paths['temp'], 
                                                os.path.basename(files['image']).replace('.fit', '.cat'))
                    }

sex_kwargs_1['config']['CATALOG_TYPE'] = 'FITS_LDAC'
#sex_kwargs_1['config']['PIXEL_SCALE'] = str(pixel_scale)
#sex_kwargs_1['config']['SEEING_FWHM'] = str(fwhm)
sex_kwargs_1['config']['GAIN'] = str(gain)
sex_kwargs_1['config']['SATUR_LEVEL'] = '50000.0'
sex_kwargs_1['config']['DETECT_THRESH'] = '5'
sex_kwargs_1['config']['ANALYSIS_THRESH'] = '5'
sex_kwargs_1['config']['FILTER'] = 'Y'
sex_kwargs_1['config']['FILTER_NAME'] = os.path.join(paths['config'], 'gauss_4.0_7x7.conv')
sex_kwargs_1['config']['CHECKIMAGE_TYPE'] = 'APERTURES,BACKGROUND,-BACKGROUND,OBJECTS,-OBJECTS,FILTERED,SEGMENTATION'
sex_kwargs_1['config']['CHECKIMAGE_NAME'] = 'check_aper.fit,check_back.fit,check_-back.fit,check_obs.fit,check_-obs.fit,check_filtered.fit,check_segm.fit'
sex_kwargs_1['temp_path'] = paths['temp']
sex_kwargs_1['params'] = ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'AWIN_IMAGE', 'BWIN_IMAGE',
                          'ERRAWIN_IMAGE','ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 
                          #'ERRA_WORLD', 'ERRB_WORLD', 
                          #'ERRTHETA_WORLD', 'X_WORLD', 'Y_WORLD', 'XWIN_WORLD', 'YWIN_WORLD', 
                          'FLUX_AUTO', 'FLUX_MAX', 'MAG_AUTO', 'FLUXERR_AUTO', 
                          # flags not supported yet (need Weightwatcher)
                          #'IMAFLAGS_ISO', 'FLAGS_WEIGHT',
                          'FLAGS', 'FLUX_RADIUS', 'ELONGATION']

# minimum catalog columns needed for SCAMP:
#XWIN_IMAGE
#YWIN_IMAGE
#ERRAWIN_IMAGE
#ERRBWIN_IMAGE
#ERRTHETAWIN_IMAGE
#ERRA_WORLD
#ERRB_WORLD
#ERRTHETA_WORLD
#X_WORLD
#Y_WORLD
#FLUX_RADIUS
#FLUX_AUTO
#FLUXERR_AUTO
#FLUX_MAX
#MAG_AUTO
#ELONGATION
#FLAGS
#AWIN_IMAGE
#BWIN_IMAGE

In [111]:
sextractor = aw.api.Astromatic(**sex_kwargs_1)

In [112]:
this_cmd, kwargs2 = sextractor.build_cmd(files['image'], **sex_kwargs_1)
print this_cmd

sex /home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images/GRB130427_R60_001_003.fit -c /home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/config/default.sex -CHECKIMAGE_NAME check_aper.fit,check_back.fit,check_-back.fit,check_obs.fit,check_-obs.fit,check_filtered.fit,check_segm.fit -PARAMETERS_NAME /home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp/sex.param -ANALYSIS_THRESH 5 -CATALOG_NAME /home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp/GRB130427_R60_001_003.cat -DETECT_THRESH 5 -FILTER Y -CATALOG_TYPE FITS_LDAC -SATUR_LEVEL 50000.0 -FILTER_NAME /home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/config/gauss_4.0_7x7.conv -CHECKIMAGE_TYPE APERTURES,BACKGROUND,-BACKGROUND,OBJECTS,-OBJECTS,FILTERED,SEGMENTATION -GAIN 4.40000009537


In [113]:
sextractor.run(files['image']) 

{'status': 'success'}

In [114]:
catalog = aw.utils.ldac.get_table_from_ldac(sex_kwargs_1['config']['CATALOG_NAME'])
catalog

NUMBER,EXT_NUMBER,XWIN_IMAGE,YWIN_IMAGE,AWIN_IMAGE,BWIN_IMAGE,ERRAWIN_IMAGE,ERRBWIN_IMAGE,ERRTHETAWIN_IMAGE,FLUX_AUTO,FLUX_MAX,MAG_AUTO,FLUXERR_AUTO,FLAGS,FLUX_RADIUS,ELONGATION
,,pix,pix,pix,pix,pix,pix,deg,ct,ct,mag,ct,,pix,
int32,int16,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32
1,1,199.168055166,38.0395543366,1.29435,1.24256,0.00895048,0.00888273,-61.3488,108010.0,6044.28,-12.5837,753.883,0,2.41308,1.42192
2,1,167.923478286,495.912756943,0.408029,0.25735,0.0281359,0.0279727,-70.1514,6337.43,3574.19,-9.50478,312.527,2,1.07645,1.24797
3,1,96.8465594062,487.20156571,1.23603,1.19444,0.0381189,0.0380669,-36.7805,18815.0,1136.28,-10.6863,506.275,0,2.15867,1.03352
4,1,484.971035118,435.943760281,1.27491,1.19144,0.0270609,0.0247532,-44.8148,13835.6,13050.4,-10.3525,305.397,0,0.407155,1.07006
5,1,436.999561041,434.086067572,0.344553,0.296117,0.0298878,0.0297697,42.8448,4538.59,2655.58,-9.1423,177.712,0,1.04116,1.70323
6,1,308.93746293,405.139739043,1.27587,1.21961,0.0210653,0.0209366,-9.62813,37558.7,2198.43,-11.4368,576.513,0,2.24851,1.07395
7,1,391.748642759,249.112014528,1.51975,0.732005,0.0861098,0.0852275,-74.556,7227.08,1640.33,-9.64741,276.569,0,2.16963,1.7267
8,1,459.813620052,224.285189293,1.33711,1.26766,0.0265599,0.0265027,-83.175,32629.5,1733.33,-11.284,582.138,0,2.36409,1.04671


### 2) SCAMP

In [115]:
scamp_kwargs_1 = {
    'config': {
        'ASTREF_CATALOG': 'UCAC-3',
        'ASTREF_BAND': 'DEFAULT',
        'SOLVE_ASTROM': 'Y',
        'SOLVE_PHOTOM': 'Y',
        'CHECKPLOT_DEV': 'NULL',
        'MOSAIC_TYPE': 'UNCHANGED',#'UNCHANGED',
        #'DISTORT_DEGREES': '1',
        'COMPUTE_PARALLAXES': 'N',
        'CROSSID_RADIUS': '3.0',
        'EXPOTIME_KEY': 'EXPTIME',
        'FULLOUTCAT_TYPE': 'FITS_LDAC',
        'FULLOUTCAT_NAME': os.path.join(paths['temp'], 'scamp_catalog.cat'),
        'PIXSCALE_MAXERR': '1.2',
        'STABILITY_TYPE': 'INSTRUMENT', #default: 'INSTRUMENT'
        'SN_THRESHOLDS': '10.0,100.0',
        'FWHM_THRESHOLDS': '0.0,10.0', #default: '0.0,100.0'
        }
}
scamp_kwargs_1['code'] = 'SCAMP'
scamp_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.scamp')

catalogs = ['temp/GRB130427_R60_001_00%d.cat' % n for n in xrange(1,4)]
catalogs = ' '.join(catalogs)
catalogs

'temp/GRB130427_R60_001_001.cat temp/GRB130427_R60_001_002.cat temp/GRB130427_R60_001_003.cat'

In [116]:
scamp = aw.api.Astromatic(**scamp_kwargs_1)

In [117]:
this_cmd, kwargs2 = scamp.build_cmd(catalogs, **scamp_kwargs_1)
print this_cmd

scamp temp/GRB130427_R60_001_001.cat temp/GRB130427_R60_001_002.cat temp/GRB130427_R60_001_003.cat -c /home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/config/default.scamp -ASTREF_BAND DEFAULT -COMPUTE_PARALLAXES N -MOSAIC_TYPE UNCHANGED -FULLOUTCAT_TYPE FITS_LDAC -CHECKPLOT_DEV NULL -CROSSID_RADIUS 3.0 -ASTREF_CATALOG UCAC-3 -EXPOTIME_KEY EXPTIME -PIXSCALE_MAXERR 1.2 -FWHM_THRESHOLDS 0.0,10.0 -SOLVE_PHOTOM Y -SOLVE_ASTROM Y -SN_THRESHOLDS 10.0,100.0 -STABILITY_TYPE INSTRUMENT -FULLOUTCAT_NAME /home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp/scamp_catalog.cat


In [306]:
scamp.run(catalogs)

{'status': 'success'}

In [118]:
# check header output after SCAMP work (need to load header from txt (.head) file)
fileobj = open('temp/GRB130427_R60_001_003.head')
hdr = fits.Header.fromtextfile(fileobj)
hdr

HISTORY   Astrometric solution by SCAMP version 2.0.4 (2014-07-09)              
COMMENT   (c) 2010-2013 IAP/CNRS/UPMC                                           
COMMENT                                                                         
EQUINOX =        2000.00000000 / Mean equinox                                   
RADESYS = 'ICRS    '           / Astrometric system                             
CTYPE1  = 'RA---TAN'           / WCS projection type for this axis              
CTYPE2  = 'DEC--TAN'           / WCS projection type for this axis              
CUNIT1  = 'DEG     '           / Axis unit                                      
CUNIT2  = 'DEG     '           / Axis unit                                      
CRVAL1  =   1.731461084210E+02 / World coordinate on this axis                  
CRVAL2  =   2.771140573907E+01 / World coordinate on this axis                  
CRPIX1  =   2.425000000000E+02 / Reference pixel on this axis                   
CRPIX2  =   2.865000000000E+

In [77]:
scamp_catalog = aw.utils.ldac.get_table_from_ldac('/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/catalogs/GRB130427_R60_001_001.cat')
scamp_catalog

NUMBER,EXT_NUMBER,XWIN_IMAGE,YWIN_IMAGE,AWIN_IMAGE,BWIN_IMAGE,ERRAWIN_IMAGE,ERRBWIN_IMAGE,ERRTHETAWIN_IMAGE,ERRA_WORLD,ERRB_WORLD,ERRTHETA_WORLD,X_WORLD,Y_WORLD,XWIN_WORLD,YWIN_WORLD,FLUX_AUTO,FLUX_MAX,MAG_AUTO,FLUXERR_AUTO,FLAGS,FLUX_RADIUS,ELONGATION
,,pix,pix,pix,pix,pix,pix,deg,deg,deg,deg,deg,deg,deg,deg,ct,ct,mag,ct,,pix,
int32,int16,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float32,float32,float32,float32,int16,float32,float32
1,1,200.001879873,37.7858236746,1.44323,1.35333,0.011065,0.0109902,-62.1096,1.45491e-05,4.84484e-06,-55.142,173.159253848,27.6448352677,173.159025736,27.645189041,1933.7,91.5996,-8.21597,15.4353,4,2.76738,1.61273
2,1,166.558903708,12.5396075255,0.884981,0.373719,0.0613923,0.0603824,42.5807,6.0186e-05,nan,-40.5507,173.136591502,27.6568747765,173.13699829,27.6566229741,89.832,32.3262,-4.88358,4.63264,4,1.49201,1.48775
3,1,384.691709513,11.2930270856,0.908017,0.418396,0.0690206,0.0679324,-56.3237,0.000135504,nan,-43.574,173.124353186,27.7278310217,173.124269547,27.727886527,93.916,21.7578,-4.93185,5.42732,4,1.50754,1.57846
4,1,413.6074166,492.082118733,1.77615,1.08217,0.180699,0.180116,72.7929,0.000171837,nan,-41.4304,173.319664854,27.4997958331,173.319508909,27.4999539796,90.8652,24.5,-4.89599,6.23148,4,2.81449,1.43413
5,1,132.398679149,491.661734178,1.15069,0.916292,0.0984648,0.0982954,-78.5888,0.000103588,1.08439e-05,-17.1235,173.210648413,27.6067080243,173.210763092,27.6066452075,102.209,27.8223,-5.02372,6.30976,4,1.98488,1.31229
6,1,97.7152617824,486.896562839,1.31941,1.19006,0.0432631,0.0431249,-35.7225,6.60725e-05,8.19749e-06,-16.9525,173.197454285,27.6146669061,173.197302405,27.6147025907,313.127,27.752,-6.2393,10.2885,4,2.25096,1.09392
7,1,431.001280447,489.984657153,0.74834,0.684469,0.0821369,0.076041,89.9702,0.0001407,4.62414e-05,-44.3769,173.323905234,27.4837813642,173.323783312,27.4840749316,82.7871,42.3496,-4.79491,5.33619,4,0.820596,1.09332
8,1,42.1091560706,485.650792118,1.12641,0.612804,0.10863,0.108103,63.123,0.000258587,1.79263e-05,-17.6974,173.190943918,27.6304435689,173.192072659,27.6300874878,58.1973,18.1211,-4.41226,3.71911,4,1.6064,2.97875


## 3) Swarp

In [119]:
files['image']
paths['temp']

'/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp'

In [120]:
coadd_fits_name = 'coadded.fits'
coadd_fits_filename = os.path.join(paths['temp'], coadd_fits_name)
swarp_kwargs_1 = {
    'config': {
        'RESAMPLE': 'N',
        'IMAGEOUT_NAME': coadd_fits_filename,
        'WEIGHTOUT_NAME': coadd_fits_filename.replace('.fits','.wtmap.fits'),
        'DELETE_TMPFILES': 'Y',
        'PROJECTION_TYPE': 'NONE',
        'CELESTIAL_TYPE ': 'NATIVE'
    }
    }

swarp_kwargs_1['code'] = 'SWarp'
swarp_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.swarp')
fits_to_coadd = ' '.join(['temp/GRB130427_R60_001_00%d.fit' % n for n in xrange(1,4)])
fits_to_coadd

'temp/GRB130427_R60_001_001.fit temp/GRB130427_R60_001_002.fit temp/GRB130427_R60_001_003.fit'

In [121]:
swarp = aw.api.Astromatic(**swarp_kwargs_1)

In [122]:
this_cmd, kwargs2 = swarp.build_cmd(fits_to_coadd, **swarp_kwargs_1)
print this_cmd

swarp temp/GRB130427_R60_001_001.fit temp/GRB130427_R60_001_002.fit temp/GRB130427_R60_001_003.fit -c /home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/config/default.swarp -RESAMPLE N -PROJECTION_TYPE NONE -WEIGHTOUT_NAME /home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp/coadded.wtmap.fits -CELESTIAL_TYPE  NATIVE -DELETE_TMPFILES Y -IMAGEOUT_NAME /home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp/coadded.fits


In [271]:
swarp.run('temp/GRB130427_R60_001_001.fit')

KeyboardInterrupt: 

### At now we have to check functions above on several images.

In [68]:
# by defualt, we search for images in paths['images']
def pipeline_1(paths):
    print "prepare..."
    # prepare
    fits_format = 'fit'
    filenames = sorted(get_fits_images_from_dir(paths['images'], files_format=fits_format))
    exposures = []
    for filename in filenames:
        exposures.append({'image': os.path.join(paths['images'], filename)})
    # STEP 1: basic extraction with SExtractor (generate catalogs) for each image
    catalog_names = []
    print 'begin SExtractor execution'
    for files in exposures:
        print files['image'] + ':'
        # Create names for the output catalogs for each image (in temp dir)
        catalog_names.append(os.path.join(paths['temp'],
            os.path.basename(files['image']).replace('.'+fits_format, '.cat')))
        # set params for sextractor execution
        sex_kwargs_1 = {'code': 'SExtractor'}
        sex_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.sex')
        sex_kwargs_1['config'] = {'CATALOG_NAME': catalog_names[-1]}
        sex_kwargs_1['config']['CATALOG_TYPE'] = 'FITS_LDAC'
        sex_kwargs_1['config']['FILTER'] = 'N'
        sex_kwargs_1['temp_path'] = paths['temp']
        sex_kwargs_1['params'] = ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'AWIN_IMAGE', 'BWIN_IMAGE',
                                  'ERRAWIN_IMAGE','ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'ERRA_WORLD', 'ERRB_WORLD', 
                                  'ERRTHETA_WORLD', 'X_WORLD', 'Y_WORLD', 'XWIN_WORLD', 'YWIN_WORLD', 
                                  'FLUX_AUTO', 'FLUX_MAX', 'MAG_AUTO', 'FLUXERR_AUTO', 
                                  'FLAGS', 'FLUX_RADIUS', 'ELONGATION']
        sextractor = aw.api.Astromatic(**sex_kwargs_1)
        print sextractor.run(files['image'])
    print 'all catalogs successfully created'
    
    # STEP 2: compute solution for extracted catalogs
    print 'SCAMP now working...'
    scamp_kwargs_1 = {
        'config': {
            'ASTREF_CATALOG': 'USNO-B1',
            'ASTREF_BAND': 'DEFAULT',
            'SOLVE_PHOTOM': 'N',
            'CHECKPLOT_DEV': 'NULL'
            }
    }
    scamp_kwargs_1['code'] = 'SCAMP'
    scamp_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.scamp')
    scamp = aw.api.Astromatic(**scamp_kwargs_1)
    this_cmd, kwargs2 = scamp.build_cmd(catalog_names, **scamp_kwargs_1)
    #print this_cmd
    print scamp.run(catalog_names)
    
    print 'SWarp now working...'
    # STEP 3: use SWarp for coadd images using .head files, computed with SCAMP
    coadd_fits_name = 'coadded.fits'
    coadd_fits_filename = os.path.join(paths['temp'], coadd_fits_name)
    swarp_kwargs_1 = {
        'config': {
            'IMAGEOUT_NAME': coadd_fits_filename,
            'WEIGHTOUT_NAME': coadd_fits_filename.replace('.fits','.wtmap.fits'),
        }
        }
    swarp_kwargs_1['code'] = 'SWarp'
    swarp_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.swarp')
    swarp = aw.api.Astromatic(**swarp_kwargs_1)
    this_cmd, kwargs2 = swarp.build_cmd([exp['image'] for exp in exposures], **swarp_kwargs_1)
    #print this_cmd
    print swarp.run([exp['image'] for exp in exposures])
    return
    # STEP 4: extract object from coadded image using PSF
    print 'SExtractor now working...'
    coadd_weight_map = coadd_fits_filename.replace('.fits','.wtmap.fits')
    sex_kwargs_2 = {
            'config': {
                'CATALOG_NAME': coadd_fits_filename.replace('.fits', '.cat'),
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': 'N',
                'WEIGHT_IMAGE': coadd_weight_map,
                'WEIGHT_TYPE': 'MAP_WEIGHT',        
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_APER(1)',
                'FLUXERR_APER(1)', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                'ELONGATION', 'VIGNET(20,20)', 'SNR_WIN']
        }
    sex_kwargs_2['code'] = 'SExtractor'
    sex_kwargs_2['config_file'] = os.path.join(paths['config'], 'default.sex')
    sex_kwargs_2['temp_path'] = paths['temp']
    sextractor = aw.api.Astromatic(**sex_kwargs_2)
    print sextractor.run(coadd_fits_filename)
    
    # Calculate PSF
    print 'PSFex now working...'
    psf_kwargs_1 = {
        'config': {
            'PHOT_APERTURES':'15'
            'CENTER_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
            'PSFVAR_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
            'PHOTFLUX_KEY': 'FLUX_AUTO',
            'PHOTFLUXERR_KEY': 'FLUXERR_AUTO',
            'CHECKPLOT_DEV': 'NULL',
            'STABILITY_TYPE':  'EXPOSURE',
            'PSF_SUFFIX': '.psf'
        }
    }
    psf_kwargs_1['code'] = 'PSFEx'
    psf_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.psfex')
    psf_kwargs_1['temp_path'] = paths['temp']
    psfex = aw.api.Astromatic(**psf_kwargs_1)
    print psfex.run(coadd_fits_filename.replace('.fits', '.cat'))
    
    # Calculate PSF photometry for coadded image
    print 'SExtractor now working...'
    output_cat_name = 'result.ldac.fits'
    catalog_name = os.path.join(paths['catalogs'], output_cat_name)
    sex_kwargs_3 = {
            'config': {
                'PSF_NAME': os.path.join(coadd_fits_filename.replace('.fits', '.psf')),
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': 'N',
                'CATALOG_NAME': catalog_name,
                'WEIGHT_IMAGE': coadd_weight_map,
                'WEIGHT_TYPE': 'MAP_WEIGHT',
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_AUTO',
                'FLUXERR_AUTO', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                'ELONGATION', 'MAG_AUTO', 'MAGERR_AUTO', 'ALPHAPSF_SKY', 'DELTAPSF_SKY',
                'ERRX2PSF_WORLD','ERRY2PSF_WORLD', 'FLUX_PSF', 'FLUXERR_PSF', 'MAG_PSF', 'MAGERR_PSF']
        }
    sex_kwargs_3['code'] = 'SExtractor'
    sex_kwargs_3['config_file'] = os.path.join(paths['config'], 'default.sex')
    sex_kwargs_3['temp_path'] = paths['temp']
    sextractor = aw.api.Astromatic(**sex_kwargs_3)
    print sextractor.run(coadd_fits_filename)
    return catalog_name

In [70]:
catalog_name = pipeline_1(paths)
#catalog = aw.utils.ldac.get_table_from_ldac(catalog_name)
#catalog

prepare...
begin SExtractor execution
/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images/GRB130427_R60_001_001.fit:
{'status': 'success'}
/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images/GRB130427_R60_001_002.fit:
{'status': 'success'}
all catalogs successfully created
SCAMP now working...
{'status': 'success'}
SWarp now working...
{'status': 'success'}


## Test pipeline components

In [29]:
def build_pipeline(pipeline, exposures, ref_catalog='IKI', ref_band='DEFAULT',
        stack_name = 'test_stack.fits', output_cat_name='test_psf.ldac.fits'):
    # Generate catalogs from sextractor
    catalog_names = []
    for files in exposures:
        # Create names for the output catalogs for each image
        catalog_names.append(os.path.join(pipeline.paths['temp'],
            os.path.basename(files['image']).replace('.fit', '.cat')))
        kwargs = {
            # image to SExtract
            'files': files,
            # Arguments to initialize Astromatic class
            'api_kwargs': {
                # Configuration parameters
                'config': {
                    'CATALOG_NAME': catalog_names[-1],
                    'CATALOG_TYPE': 'FITS_LDAC',
                    'FILTER': 'N',
                    #'WEIGHT_TYPE': 'MAP_WEIGHT',
                },
                # Output parameters
                'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                    'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_AUTO',
                    'FLUXERR_AUTO', 'IMAFLAGS_ISO', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                    'ELONGATION'],
            },
        }
        # Add the step to the pipeline
        pipeline.add_step(aw.api.run_sex, ['step1', 'SExtractor'], **kwargs)

    # Use SCAMP to get astrometric solutions
    kwargs = {
        'catalogs': catalog_names,
        'api_kwargs': {
            'config': {
                'ASTREF_CATALOG': ref_catalog,
                'ASTREF_BAND': ref_band,
                'SOLVE_PHOTOM': 'N',
                'CHECKPLOT_DEV': 'NULL'
            },
        }
    }
    pipeline.add_step(aw.api.run_scamp, ['step2', 'SCAMP'],**kwargs)

    # Resample (rotate and scale) and combine (stack) images using SWarp
    stack_filename = os.path.join(pipeline.paths['temp'], stack_name)
    kwargs = {
        'filenames': [exp['image'] for exp in exposures],
        'api_kwargs': {
            'config': {
                'WEIGHT_TYPE': 'MAP_WEIGHT',
                'WEIGHT_SUFFIX': '.wtmap.fits',
                'IMAGEOUT_NAME': stack_filename,
                'WEIGHTOUT_NAME': stack_filename.replace('.fits','.wtmap.fits'),
            },
        }
    }
    pipeline.add_step(aw.api.run_swarp, ['step3', 'SWarp'], **kwargs)

    # Get positions in stack for PSF photometry (SExtractor -> PSFex -> SExtractor)
    kwargs = {
        'files': {
            'image': stack_filename,
            'wtmap': stack_filename.replace('.fits', '.wtmap.fits')
        },
        'api_kwargs': {
            'config': {
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': 'N',
                'WEIGHT_TYPE': 'MAP_WEIGHT',
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_APER(1)',
                'FLUXERR_APER(1)', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                'ELONGATION', 'VIGNET(20,20)', 'SNR_WIN'],
        }
    }
    pipeline.add_step(aw.api.run_sex, ['step4', 'SExtractor'], **kwargs)

    # Calculate PSF
    kwargs = {
        'catalogs': stack_filename.replace('.fits', '.cat'),
        'api_kwargs': {
            'config': {
                'CENTER_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
                'PSFVAR_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
                'CHECKPLOT_DEV': 'NULL',
                'PSF_SUFFIX': '.psf'
            }
        }
    }
    pipeline.add_step(aw.api.run_psfex, ['step5', 'PSFEx'], **kwargs)

    # Calculate PSF photometry for coadded image
    catalog_name = os.path.join(pipeline.paths['catalogs'], output_cat_name)
    kwargs = {
        'files': {
            'image': stack_filename,
            'wtmap': stack_filename.replace('.fits', '.wtmap.fits')
        },
        'api_kwargs': {
            'config': {
                'PSF_NAME': os.path.join(stack_filename.replace('.fits', '.psf')),
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': 'N',
                'CATALOG_NAME': catalog_name,
                'WEIGHT_TYPE': 'MAP_WEIGHT',
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_AUTO',
                'FLUXERR_AUTO', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                'ELONGATION', 'MAG_AUTO', 'MAGERR_AUTO', 'ALPHAPSF_SKY', 'DELTAPSF_SKY',
                'ERRX2PSF_WORLD','ERRY2PSF_WORLD', 'FLUX_PSF', 'FLUXERR_PSF', 'MAG_PSF', 'MAGERR_PSF'],
        }
    }
    pipeline.add_step(aw.api.run_sex, ['step6', 'SExtractor'], **kwargs)

    def save_output(pipeline, old_stack, new_stack, old_cat, new_cat):
        # Copy the final stack and catalog from the temp folder
        import shutil
        # Move the weight map if it exists
        if os.path.isfile(old_stack.replace('.fits', '.wtmap.fits')):
            shutil.move(old_stack.replace('.fits', '.wtmap.fits'), new_stack.replace('.fits', '.wtmap.fits'))
        shutil.move(old_stack, new_stack)
        shutil.move(old_cat, new_cat)
        result = {
            'status': 'success'
        }
        return result

    kwargs = {
        'old_stack': stack_filename,
        'new_stack': os.path.join(pipeline.paths['stacks'], stack_name),
        'old_cat': catalog_name,
        'new_cat': os.path.join(pipeline.paths['catalogs'], output_cat_name)
    }
    pipeline.add_step(save_output, ['step7', 'save_output'], **kwargs)

    return pipeline